Testing a new calc for players selection using team means for matches home/guest and also position means. This method simplify the cartoleiro class method.

In [15]:
import cartoleiro

cart = cartoleiro.Cartoleiro(True)
df_scouts = cart.scout_table
df_rounds = cart.rounds_table
df_teams = cart.read_teams()

import pandas as pd

df_temp = df_scouts[["rodada_id", "atleta_id", "clube_id", "apelido", "posicao_id", "pontos_num", "jogos_num", "preco_num"]]
ind = df_temp.set_index(["atleta_id", "rodada_id"])
df_players = pd.DataFrame()
for player in ind.index.get_level_values(0).unique():
    # .copy(deep=True) to supress SettingWithCopyWarning
    df = df_temp[df_temp.atleta_id == player].copy(deep=True) 
    # test to flag the player who plays in a specific round
    df["jogou"] = (df["jogos_num"].diff() != 0) & (df["jogos_num"] != 0)
    df_players = pd.concat([df_players, df], axis=0)
# discard rows where player didnt play the match
df_scouts = df_players[df_players.jogou]

df_scouts = df_scouts.merge(df_teams[["id", "abreviacao"]], left_on="clube_id", right_on="id")

home = df_rounds[["rodada_id", "clube_casa_id"]].copy(deep=True) 
home.loc[:,"home_guest"] = len(home)*["H"]
df_scouts = df_scouts.merge(home, how="left", left_on=["rodada_id", "clube_id"], 
                            right_on=["rodada_id", "clube_casa_id"])
df_scouts["home_guest"] = df_scouts["home_guest"].fillna("G")

df_scouts = df_scouts.drop(["id", "jogou", "clube_casa_id"], axis=1)

df_scouts.head()
#df_scouts[["pontos_num"]].agg(["count","mean", "std", "min", "median", "max"])


,rodada_id,atleta_id,clube_id,apelido,posicao_id,pontos_num,jogos_num,preco_num,abreviacao,home_guest
0,1,81091,282,Jair,4,1.5,1,6.59,ATL,H
1,2,81091,282,Jair,4,2.0,2,6.07,ATL,G
2,3,81091,282,Jair,4,5.7,3,7.20,ATL,G
3,6,81091,282,Jair,4,2.5,4,6.73,ATL,G
4,10,81091,282,Jair,4,6.0,5,7.16,ATL,G


In [16]:
df_mean_pos = df_scouts.groupby(["posicao_id", "home_guest"])["pontos_num"].agg(["mean"])
df_mean_pos = df_mean_pos.drop(6) 
df_mean_pos = df_mean_pos.reset_index()
df_mean_pos.columns.values[2] = "idx_pos"
df_mean_pos

#df_mean_pos.pivot(index="posicao_id", columns="home_guest", values="mean").drop(6)



,posicao_id,home_guest,idx_pos
0,1,G,4.416489
1,1,H,4.229570
2,2,G,2.997236
3,2,H,3.888482
4,3,G,2.302764
5,3,H,3.440365
6,4,G,2.131118
7,4,H,2.738373
8,5,G,2.362021
9,5,H,3.665038


In [17]:
from cartolafc_api import cartola_api

df_market_min_matches = (pd.DataFrame(cartola_api.read_data()["atletas"]))
df_market = df_market_min_matches[["atleta_id", "status_id", "preco_num"]]
df_market = df_market[df_market.status_id == 7].drop("status_id", axis=1)

df_market_min_matches = df_market_min_matches[df_market_min_matches.jogos_num >= 5]
df_market_min_matches = df_market_min_matches[df_market_min_matches.status_id == 7]


df_market.head()


,atleta_id,preco_num
2,42222,5.87
3,42116,6.94
4,37246,8.92
6,37281,9.66
7,36940,7.52


In [18]:
cart.read_next_round()
next_round = cart.next_round
next_round = next_round[next_round["valida"] == True]
home_teams = list(next_round["clube_casa_id"])
guest_teams = list(next_round["clube_visitante_id"])

home_teams


[285, 283, 356, 264, 277, 267, 263, 314, 290, 341]

In [19]:
df_team_mean = df_scouts[df_scouts.atleta_id.isin(df_market["atleta_id"])].groupby([
    "clube_id","home_guest"])["pontos_num"].agg(["mean","median"])
df_team_mean = df_team_mean.reset_index()
df_team_mean = df_team_mean.pivot(index="clube_id", columns="home_guest", values="mean")
df_team_mean["idx_round"] = None
for team in home_teams: 
    df_team_mean.loc[team, "idx_round"] = df_team_mean.loc[team, "H"] 
for team in guest_teams: 
    df_team_mean.loc[team, "idx_round"] = df_team_mean.loc[team, "G"] 
        
df_team_mean = df_team_mean.drop(["G", "H"], axis=1).reset_index()
df_team_mean

home_guest,clube_id,idx_round
0,262,5.51062
1,263,2.66861
2,264,5.14567
3,265,4.68886
4,266,2.55275
5,267,3.66188
6,275,2.5314
7,276,2.66938
8,277,5.48333
9,282,2.87816


In [20]:
df_players_to_sel = df_scouts[df_scouts.atleta_id.isin(df_market_min_matches["atleta_id"])].groupby([
    "clube_id","atleta_id","posicao_id", "apelido", "abreviacao"])["pontos_num"].agg(["mean","median"])


df_players_to_sel["home_guest"] = None

for team in df_players_to_sel.index.get_level_values(0).unique():
    if team in home_teams:
        df_players_to_sel.loc[team,"home_guest"] = "H"
    else:
        df_players_to_sel.loc[team,"home_guest"] = "G"
        
df_players_to_sel = df_players_to_sel.reset_index()
df_players_to_sel = df_players_to_sel.merge(df_mean_pos, how="left", on=["posicao_id", "home_guest"])
df_players_to_sel = df_players_to_sel.merge(df_team_mean, how="left", on="clube_id")
df_players_to_sel = df_players_to_sel.merge(df_market[["atleta_id", "preco_num"]], how="left", on="atleta_id")
df_players_to_sel["idx_player"] = pd.to_numeric((1 * df_players_to_sel["mean"] + 2 * df_players_to_sel["median"] +
                                   df_players_to_sel["idx_pos"] + 2 * df_players_to_sel["idx_round"]) / 6)
df_players_to_sel.head()

,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
0,262,38144,2,Rafinha,FLA,5.187500,6.00,G,2.997236,5.51062,10.42,5.200994
1,262,38509,1,Diego Alves,FLA,5.160000,5.00,G,4.416489,5.51062,7.88,5.099620
2,262,38750,2,Filipe Luís,FLA,5.266667,4.85,G,2.997236,5.51062,13.19,4.830856
3,262,71224,6,Jorge Jesus,FLA,6.042000,6.29,G,NaN,5.51062,14.73,NaN
4,262,73800,3,Rodrigo Caio,FLA,3.938462,3.80,G,2.302764,5.51062,8.82,4.143743


In [21]:
print("GOLEIROS")

df_gk = df_players_to_sel[df_players_to_sel.posicao_id == 1]
df_gk.sort_values("idx_player", ascending=False).head(3)

GOLEIROS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
38,265,78584,1,Douglas Friedrich,BAH,7.733333,8.75,G,4.416489,4.68886,13.97,6.504591
176,341,86776,1,Jordi,CSA,6.405556,6.95,H,4.229570,3.231,10.66,5.166188
1,262,38509,1,Diego Alves,FLA,5.160000,5.00,G,4.416489,5.51062,7.88,5.099620


In [22]:
print("ZAGUEIROS")

df_bw = df_players_to_sel[df_players_to_sel.posicao_id == 3]
df_bw.sort_values("idx_player", ascending=False).head(7)

ZAGUEIROS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
127,285,79113,3,Víctor Cuesta,INT,5.930769,5.00,H,3.440365,5.86143,14.51,5.182332
58,267,38505,3,Leandro Castan,VAS,5.712500,6.60,H,3.440365,3.66188,9.59,4.946105
123,285,63354,3,Rodrigo Moledo,INT,4.471429,5.00,H,3.440365,5.86143,6.55,4.939108
11,262,105897,3,Pablo Marí,FLA,5.637500,5.30,G,2.302764,5.51062,7.54,4.926916
27,264,69014,3,Manoel,COR,4.460000,5.50,H,3.440365,5.14567,10.89,4.865283
93,277,93368,3,Lucas Veríssimo,SAN,4.594118,4.90,H,3.440365,5.48333,9.54,4.800191
106,283,60819,3,Dedé,CRU,5.271429,5.35,H,3.440365,4.46529,15.90,4.723730


In [31]:
print("ATACANTES")

df_fw = df_players_to_sel[df_players_to_sel.posicao_id == 5]
df_fw.sort_values("idx_player", ascending=False).head(7)

ATACANTES


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
6,262,83257,5,Gabriel,FLA,10.773333,9.00,G,2.362021,5.51062,20.99,7.026098
69,267,105786,5,Talles Magno,VAS,5.666667,8.90,H,3.665038,3.66188,3.23,5.742579
88,277,70360,5,Eduardo Sasha,SAN,6.544444,5.75,H,3.665038,5.48333,17.52,5.446025
126,285,75295,5,Paolo Guerrero,INT,5.037500,4.60,H,3.665038,5.86143,12.65,4.937566
101,282,101715,5,Alerrandro,ATL,6.375000,6.35,G,2.362021,2.87816,7.78,4.532223
9,262,90285,5,Bruno Henrique,FLA,6.300000,3.50,G,2.362021,5.51062,19.29,4.447209
34,265,61188,5,Gilberto,BAH,7.333333,3.70,G,2.362021,4.68886,16.91,4.412179


In [30]:
print("LATERAIS")

df_sw = df_players_to_sel[df_players_to_sel.posicao_id == 2]
df_sw.sort_values("idx_player", ascending=False).head(5)

LATERAIS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
92,277,88065,2,Jorge,SAN,6.760000,7.2,H,3.888482,5.48333,14.34,6.002525
113,284,68821,2,Rafael Galhardo,GRE,6.257143,7.4,G,2.997236,3.60039,5.49,5.209194
0,262,38144,2,Rafinha,FLA,5.187500,6.0,G,2.997236,5.51062,10.42,5.200994
25,264,42500,2,Fagner,COR,5.784615,5.6,H,3.888482,5.14567,12.92,5.194072
41,265,90444,2,Moisés,BAH,5.400000,6.6,G,2.997236,4.68886,12.82,5.162493


In [29]:
print("MEIAS")

df_mf = df_players_to_sel[df_players_to_sel.posicao_id == 4]
df_mf.sort_values("idx_player", ascending=False).head(7)

MEIAS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
7,262,87863,4,Arrascaeta,FLA,11.618182,11.60,G,2.131118,5.51062,16.23,7.995088
137,290,90572,4,Léo Sena,GOI,5.328571,6.30,H,2.738373,3.94375,8.72,4.759074
8,262,89256,4,Gerson,FLA,3.844444,5.30,G,2.131118,5.51062,8.82,4.599466
44,265,92981,4,Ronaldo,BAH,5.450000,5.20,G,2.131118,4.68886,5.57,4.559807
125,285,72362,4,Rodrigo Lindoso,INT,5.527273,3.30,H,2.738373,5.86143,11.35,4.431417
182,354,71844,4,Thiago Galhardo,CEA,6.887500,6.05,G,2.131118,2.58549,11.46,4.381601
32,264,98706,4,Pedrinho,COR,5.123077,3.90,H,2.738373,5.14567,11.53,4.325464


In [26]:
#df_players_to_sel.info()
print("TÉCNICOS")
df_players_to_sel.groupby(["clube_id", "abreviacao"])["idx_player"].agg(["mean"]).sort_values("mean", ascending=False).head(5)



TÉCNICOS


,,mean
clube_id,abreviacao,
262,FLA,5.045768
285,INT,4.622858
277,SAN,4.548998
264,COR,4.097967
265,BAH,3.900459


In [34]:
print("CAPITÃES")
df_captains = df_fw
#df_captains = df_captains.append(df_gk)
#df_captains = df_captains.append(df_bw)
df_captains = df_captains.append(df_mf)
#df_captains = df_captains.append(df_sw)
df_captains[df_captains.home_guest == "H"].sort_values("median", ascending=False).head(5)
            

CAPITÃES


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
69,267,105786,5,Talles Magno,VAS,5.666667,8.90,H,3.665038,3.66188,3.23,5.742579
137,290,90572,4,Léo Sena,GOI,5.328571,6.30,H,2.738373,3.94375,8.72,4.759074
88,277,70360,5,Eduardo Sasha,SAN,6.544444,5.75,H,3.665038,5.48333,17.52,5.446025
66,267,92496,4,Raul,VAS,4.987500,4.80,H,2.738373,3.66188,9.06,4.108274
126,285,75295,5,Paolo Guerrero,INT,5.037500,4.60,H,3.665038,5.86143,12.65,4.937566


In [35]:
df_scouts[df_scouts.atleta_id == 70360]

,rodada_id,atleta_id,clube_id,apelido,posicao_id,pontos_num,jogos_num,preco_num,abreviacao,home_guest
2017,1,70360,277,Eduardo Sasha,5,13.4,1,12.77,SAN,G
2018,2,70360,277,Eduardo Sasha,5,13.1,2,18.80,SAN,H
2019,3,70360,277,Eduardo Sasha,5,0.9,3,16.89,SAN,G
2020,4,70360,277,Eduardo Sasha,5,3.9,4,16.99,SAN,H
2021,6,70360,277,Eduardo Sasha,5,1.2,5,15.90,SAN,H
2022,7,70360,277,Eduardo Sasha,5,9.4,6,17.26,SAN,G
2023,8,70360,277,Eduardo Sasha,5,9.2,7,17.64,SAN,H
2024,9,70360,277,Eduardo Sasha,5,9.7,8,18.02,SAN,H
2025,10,70360,277,Eduardo Sasha,5,0.0,9,16.48,SAN,G
2026,11,70360,277,Eduardo Sasha,5,1.2,10,16.46,SAN,G
